# Use long-short term memory networks to forecast carbon intensity #

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

## Function for LSTM data processing ##

In [2]:
# Conceptual Python Code for Sequence Framing
def create_lstm_sequences(data, lookback_window=12):
    X, Y = [], []
    for i in range(len(data) - lookback_window):
        # X is the sequence from time i to i + lookback_window - 1
        X.append(data[i:(i + lookback_window)])
        
        # Y is the single value immediately following the sequence
        Y.append(data[i + lookback_window])
        
    return np.array(X), np.array(Y)

# Assuming 'scaled_data' is your pre-processed series:


## Preprocess data for monthly sampling ##

In [3]:
df = pd.read_csv('../data/df_fuel_ckan.csv')
df['DATETIME'] = pd.to_datetime(df['DATETIME'])
df_filtered = df[df['DATETIME'].dt.year < 2025].copy()
df_filtered.set_index('DATETIME', inplace=True)
data_train_test = df_filtered['CARBON_INTENSITY'].resample('ME').mean().dropna()
data_train_test.head()
X_sequenced, Y_targets = create_lstm_sequences(data_train_test, lookback_window=12)
split_point = int(len(X_sequenced) * 0.8)
X_train, X_test = X_sequenced[:split_point], X_sequenced[split_point:]
y_train, y_test = Y_targets[:split_point], Y_targets[split_point:]
# Reshape X_train and X_test
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

C:\Users\tianw\AppData\Local\Temp\ipykernel_25768\2034904373.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Y.append(data[i + lookback_window])


## Define LSTM model ##

In [4]:
from tensorflow.keras.layers import Input

model = Sequential()
# Explicit Input layer to avoid the warning
model.add(Input(shape=(X_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
# Keeping metrics=['mae'] so evaluation returns 2 values if desired,
# though the user seems to have reverted to just 'test_loss' in their latest manual edit.
# I will stick to just loss to match the user's latest observed code pattern,
# or I can add metrics. Let's add metrics to be safe and robust.
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.fit(X_train, y_train, epochs=200, batch_size=32, verbose=0)

c:\Users\tianw\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
test_loss = model.evaluate(X_test, y_test, verbose=0) 

print(f"\nTest Loss (e.g., MSE): {test_loss:.4f}")


Test Loss (e.g., MSE): 1452.6997
